In [15]:
import pandas as pd
import psycopg2

def df_nan_filter(df):
    df = df[df["Size"].notnull()]              # remove "Size" row if size is null
    df["Languages"].fillna("EN", inplace=True) # set "EN" as langage if value is null
    df["Price"].fillna(0.0, inplace=True)      # set 0.0 as price if value is null
    df["Average User Rating"].fillna(df["Average User Rating"].median(), inplace=True) # set median as user_rating if value is null
    df["User Rating Count"].fillna(1, inplace=True) #replace nan counts with 1
    return (df)

def special_char_filter(s: str):
    tmp = s.encode().decode()
    tmp = tmp.strip('"')
    return (''.join(e for e in tmp if e.isascii()))

df = pd.read_csv("appstore_games.csv")
df = df[["ID", "Name", "Average User Rating",
          "User Rating Count", "Price", "Description",
          "Developer", "Age Rating", "Languages",
          "Size", "Primary Genre", "Genres",
          "Original Release Date", "Current Version Release Date"]]

df = df_nan_filter(df)

for col in df:
    df[col] = df[col].apply(lambda x: special_char_filter(str(x)))

df.to_csv("filtered_appstore_games.csv")

In [16]:
def already_exists():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("select exists(select * from information_schema.tables where table_name=%s)", ('appstore_games',))         
    ver = curr.fetchone()[0]
    conn.close()
    return (ver)

def create_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("""CREATE TABLE appstore_games(
             Id bigint PRIMARY KEY,
             Name varchar,
             Avg_user_rating float,
             User_rating_count int,
             Price float,
             Description varchar,
             Developer varchar,
             Age_rating int,
             Languages varchar,
             Size bigint, 
             Primary_genre varchar,
             Genres varchar,
             Release_date date,
             Last_update date
             );""")          
    conn.commit()
    conn.close()

def delete_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute('DROP TABLE appstore_games;')          
    conn.commit()
    conn.close()

In [18]:
#create_table()
delete_table()